In [2]:
# setup
import numpy as np
import cv2
import pprint
import os
from matplotlib import pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
# set root directory
ROOT_DIR = os.path.abspath(os.curdir)

# workflow
### 1. load data
### 2. normalize
### 3. extract features
### 4. classify

In [3]:
# load face data
from util.data_utils import load_data
X_train, y_train, X_train_origin, X_train_face_cords, \
X_test, y_test, X_test_origin, X_test_face_cords = load_data(verbose=False)

In [3]:
# sanity check
print(X_train.shape)
print(y_train.shape)
print(X_train_origin.shape)
print(X_train_face_cords.shape)
print(X_test.shape)
print(y_test.shape)
print(X_test_origin.shape)
print(X_test_face_cords.shape)

(140,)
(140,)
(140, 576, 768, 3)
(140, 4)
(104,)
(104,)
(104, 576, 768, 3)
(104, 4)


## LBP feature

In [43]:
# for normalizing, calculate meidian of width, height of images
width = []
height = []
for img in X_train:
    width.append(img.shape[1])
    height.append(img.shape[0])
median_size = (int(np.around(np.median(width))), int(np.around(np.median(height))))

In [84]:
# Arbitrary set image size
median_size = (256, 256)

In [85]:
# normalize input data
from util.data_utils import normalize_image
for i, img in enumerate(X_train):
    X_train[i] = normalize_image(img, median_size)
for i, img in enumerate(X_test):
    X_test[i] = normalize_image(img, median_size)

In [43]:
# extract features from train images
from features.Lbp import extract_feature_lbp
X_train_feats = extract_feature_lbp(X_train, block_size = 3, verbose=True)

10 featurizing completed.
20 featurizing completed.
30 featurizing completed.
40 featurizing completed.
50 featurizing completed.
60 featurizing completed.
70 featurizing completed.
80 featurizing completed.
90 featurizing completed.
100 featurizing completed.
110 featurizing completed.
120 featurizing completed.
130 featurizing completed.
140 featurizing completed.
All featurizing completed.


In [44]:
# extract features of test images
X_test_feats = extract_feature_lbp(X_test, block_size = 3, verbose=True)

10 featurizing completed.
20 featurizing completed.
30 featurizing completed.
40 featurizing completed.
50 featurizing completed.
60 featurizing completed.
70 featurizing completed.
80 featurizing completed.
90 featurizing completed.
100 featurizing completed.
104 featurizing completed.
All featurizing completed.


## HOG feature

In [141]:
# recommended size : 128x64
median_size = (128, 64)
from features.Hog import *

In [142]:
# normalize input size
from util.data_utils import normalize_image
for i, img in enumerate(X_train):
    X_train[i] = normalize_image(img, median_size)
for i, img in enumerate(X_test):
    X_test[i] = normalize_image(img, median_size)

In [143]:
# extract features from train images 
X_train_feats = extract_feature(X_train)

In [144]:
# extract features from test images
X_test_feats = extract_feature(X_test)

# SVM Classifier

In [145]:
from classifiers.svm import SVM
# convert names of y_train, y_test labels into integer
y_train_num = [ int(val[-1:]) for val in y_train]
y_test_num = [ int(val[-1:]) for val in y_test]
svm = SVM()
# train svm
#params = [0, 2, 1, 0.01]
svm.train(X_train_feats, y_train_num)#, params)

In [146]:
# predict train data
y_train_pred = svm.predict(X_train_feats, y_train_num)

Accuracy = 100% (140/140) (classification)
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]


In [147]:
# predict test data
y_test_pred = svm.predict(X_test_feats, y_test_num)

Accuracy = 86.5385% (90/104) (classification)
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 1.0, 3.0, 3.0, 3.0, 6.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 4.0, 6.0, 6.0, 6.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 4.0, 4.0, 4.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 6.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 6.0, 7.0, 7.0, 7.0, 7.0, 7.0, 3.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0]


In [163]:
# visualizing result
from util.data_utils import visualize_pred
y_svm_pred_ids = ["ID_" + str(int(i)) for i in y_test_pred]
visualize_pred(X_test_face_cords, X_test_origin, y_svm_pred_ids, y_test)

# Nearest Neighbor Classifier

In [148]:
from classifiers.nearest_neighbor import NearestNeighbor
nn = NearestNeighbor(X_train_feats, y_train)

In [149]:
# predict train data
y_train_pred = nn.predict(X_train_feats)
accuracy = np.mean(y_train_pred == y_train)
print(accuracy)

1.0


In [150]:
# predict test data
y_test_pred = nn.predict(X_test_feats)
accuracy = np.mean(y_test_pred == y_test)
print(accuracy)

0.9326923076923077


In [168]:
# visualizing
from util.data_utils import visualize_pred
y_nn_pred_ids = y_test_pred
visualize_pred(X_test_face_cords, X_test_origin, y_nn_pred_ids, y_test)